# 네이버 영화 리뷰 감성 분류

* 네이버 영화 평점 데이터 : https://github.com/e9t/nsmc
    * 네이버 영화 사이트에 있는 리뷰 20만 개를 모은 데이터.
* 20만 개의 데이터 중 15만 개는 훈련 데이텟으로 ratings_train.txt 파일에 저장되어 있고 5만 개는 ratings_test.txt 파일에 저장.
* 부정 리뷰는 1~4까지 점수, 긍정 리뷰는 6~10까지 점수를 매긴 리뷰
    * 훈련 데이터셋과 테스트 데이터셋의 부정과 긍정 리뷰는 약 50%씩 구성.

* 한글 텍스트 처리
    * 한글은 영어와 달리 조사와 어미가 발달.
    * BoW나 어간 추출보다 **표제어 추출 방식**이 적합.
        * 형태소 분석

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 한글 형태소 분석을 위한 패키지
!pip install konlpy soynlp

In [3]:
!pip install tweepy==3.10

In [4]:
import konlpy
import pandas as pd
import numpy as np

`read_csv()`는 기본적으로 콤마를 기준으로 필드를 구분한다. 그래서 `delimiter='\t'`으로 지정하여 탭으로 변경. 기본적으로 판다스는 빈 문자열을 NaN으로 인식한다. 빈 문자열을 그대로 유지하기 위해 `keep_default_na` 매개변수를 False로 지정. 

In [5]:
df_train = pd.read_csv('/content/drive/MyDrive/머신러닝 교과서 with 파이썬, 사이킷런, 텐서플로/data/ratings_train.txt',
                 delimiter='\t', keep_default_na=False)
df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
df_train['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  150000 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [8]:
X_train = df_train['document'].values
y_train = df_train['label'].values

In [9]:
df_test = pd.read_csv('/content/drive/MyDrive/머신러닝 교과서 with 파이썬, 사이킷런, 텐서플로/data/ratings_test.txt',
                 delimiter='\t', keep_default_na=False)
X_test = df_test['document'].values
y_test = df_test['label'].values

In [10]:
print(len(X_train), np.bincount(y_train))
print(len(X_test), np.bincount(y_test))

150000 [75173 74827]
50000 [24827 25173]


* konlpy
    * 다섯 개의 한국어 형태소 분석기를 파이썬 클래스로 감싸서 제공하는 래퍼 패키지.
    * https://konlpy.org/ko/latest/
    * 스칼라로 개발된 open-korean-text 한국어 처리기를 제공하는 `Okt` 클래스

`konlpy.tag` 패키지에서 `Okt` 클래스를 임포트하고 객체를 만든 다음 훈련 데이터셋에 있는 문장 하나를 `morphs()` 메서드로 형태소로 나눈다.

In [11]:
from konlpy.tag import Okt

okt = Okt()
print(X_train[4])
print(okt.morphs(X_train[4]))

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']


한글 문장에서 조사와 어미가 잘 구분되어 출력되었다. '사이몬페그'와 같은 고유 명사 처리는 완벽하지 않다.

`TfidfVectorizer`을 `ngram_range=(1, 2)`로 설정하여 유니그램과 바이그램을 사용하고 `min_df=3`으로 지정하여 3회 미만으로 등장하는 토큰은 무시한다. 또한 `max_df=0.9`로 두어 가장 많이 등장하는 상위 10%의 토큰도 무시한다. 이러한 작업이 불용어로 생각할 수 있는 토큰을 제거할 것이다.

* 참고 : 토큰 데이터를 생성하는 시간이 많이 걸리므로 주피터 노트북에서는 다음 번 실행 때 이 과정을 건너뛸 수 있도록 이 데이터를 한 번 생성하여 npz 파일로 저장한다.

In [12]:
import os
from scipy.sparse import save_npz, load_npz
from sklearn.feature_extraction.text import TfidfVectorizer

if not os.path.isfile('okt_train.npz'):
    tfidf = TfidfVectorizer(ngram_range=(1, 2), 
                            min_df=3,
                            max_df=0.9,
                            tokenizer=okt.morphs, 
                            token_pattern=None)
    tfidf.fit(X_train)
    X_train_okt = tfidf.transform(X_train)
    X_test_okt = tfidf.transform(X_test)
    save_npz('okt_train.npz', X_train_okt)
    save_npz('okt_test.npz', X_test_okt)
else:
    X_train_okt = load_npz('okt_train.npz')
    X_test_okt = load_npz('okt_test.npz')

* `X_train_okt`와 `X_test_okt`가 준비되었으므로 `SGDClassifier` 클래스를 사용해 감성 분류.

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.utils.fixes import loguniform

sgd = SGDClassifier(loss='log', random_state=1) # 손실 함수는 로지스틱 손실('log')을 사용, 다른 손실함수를 매개변수 탐색에 포함할 수 있다.
param_dist = {'alpha':loguniform(0.0001, 100.0)} # 규제를 위한 매개변수, RandomizedSearchCV 클래스를 사용하기 위해 loguniform 함수로 탐색 번위 지정

rsv_okt = RandomizedSearchCV(estimator=sgd,
                             param_distributions=param_dist,
                             n_iter=50, # 총 반복횟수는 50회
                             random_state=1,
                             verbose=1)
rsv_okt.fit(X_train_okt, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc2bca3b050>},
                   random_state=1, verbose=1)

In [14]:
print(rsv_okt.best_score_) # 최상의 점수
print(rsv_okt.best_params_) # 최상의 매개변수
rsv_okt.score(X_test_okt, y_test) # 테스트 데이터 점수

0.8251533333333334
{'alpha': 0.0001001581395585897}


0.8189

* soynlp
    * 파이썬으로 구현된 형태소 분석 패키지
    * https://github.com/lovit/soynlp
    * 3개의 토큰화 클래스 제공
        * `LToeknizer` : 띄어쓰기가 잘 되어 있을 때
        * `MaxScoreTokenizer` : 띄어쓰기가 되어 있지 않는 긴 문자열일 때
        * `RegexTokenizer` : 규칙 기반으로 단어열만 들 수 있다


In [15]:
from soynlp.tokenizer import LTokenizer

lto = LTokenizer()
print(lto.tokenize(X_train[4]))

['사이몬페그의', '익살스런', '연기가', '돋보였던', '영화!스파이더맨에서', '늙어보이기만', '했던', '커스틴', '던스트가', '너무나도', '이뻐보였다']


* soynlp는 말뭉치의 통계 데이터를 기반으로 동작하기 때문에 기본 LTokenizer 객체로는 공백으로만 토큰화를 수행한다.

In [16]:
# LTokenizer에 필요한 통계 데이터를 생성하기 위해 WordExtractor를 사용
from soynlp.word import WordExtractor

# WordExtractor 객체를 만든 후 train() 메서드에 X_train을 전달하여 훈련.
# 끝나면 word_scores() 메서드에서 단어의 점수를 얻을 수 있다.
# 반환된 scores 객체는 단어마다 결합 점수(cohesion score)와 브랜칭 엔트로피(branching entropy)를 가진 딕셔너리이다.
word_ext = WordExtractor()
word_ext.train(X_train)
scores = word_ext.word_scores()

training was done. used memory 1.377 Gb
all cohesion probabilities was computed. # words = 85683
all branching entropies was computed # words = 101540
all accessor variety was computed # words = 101540


`soynlp` 깃허브의 튜토리얼 (https://github.com/lovit/soynlp/blob/master/tutorials/wordextractor_lecture.ipynb) 을 따라 결합 점수(`cohesion_forward`)와 브랜칭 엔트로피(`right_branching_entropy`)에 지수를 취한 값에 곱하여 최종 점수를 만든다.

In [17]:
import math

score_dict = {key: scores[key].cohesion_forward * 
              math.exp(scores[key].right_branching_entropy)
              for key in scores}

In [18]:
# 이 점수를 LTokenizer의 scores 매개변수로 전달하여 객체를 만들고 위에서 테스트한 샘플에 다시 적용
lto = LTokenizer(scores=score_dict)
print(lto.tokenize(X_train[4]))

['사이', '몬페그의', '익살스', '런', '연기', '가', '돋보', '였던', '영화', '!스파이더맨에서', '늙어', '보이기만', '했던', '커스틴', '던스트가', '너무', '나도', '이뻐', '보였다']


`lto.toeknizer` 메서드를 `TfidfVectorizer` 클래스에 전달하여 `konlpy`를 사용했을 때와 같은 조건으로 훈련 데이터셋과 테스트 데이터셋을 변환.

In [19]:
if not os.path.isfile('soy_train.npz'):
    tfidf = TfidfVectorizer(ngram_range=(1, 2),
                            min_df=3,
                            max_df=0.9,
                            tokenizer=lto.tokenize, 
                            token_pattern=None)
    tfidf.fit(X_train)
    X_train_soy = tfidf.transform(X_train)
    X_test_soy = tfidf.transform(X_test)
    save_npz('soy_train.npz', X_train_soy)
    save_npz('soy_test.npz', X_test_soy)
else:
    X_train_soy = load_npz('soy_train.npz')
    X_test_soy = load_npz('soy_test.npz')

In [20]:
rsv_soy = RandomizedSearchCV(estimator=sgd,
                             param_distributions=param_dist,
                             n_iter=50,
                             random_state=1,
                             verbose=1)
rsv_soy.fit(X_train_soy, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc2bca3b050>},
                   random_state=1, verbose=1)

In [21]:
print(rsv_soy.best_score_)
print(rsv_soy.best_params_)
rsv_soy.score(X_test_soy, y_test)

0.8141066666666665
{'alpha': 0.0001001581395585897}


0.8085

### 데이터 전처리


In [22]:
df_train = pd.read_csv('/content/drive/MyDrive/머신러닝 교과서 with 파이썬, 사이킷런, 텐서플로/data/ratings_train.txt', delimiter='\t')

In [23]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [24]:
df_train[df_train.isnull().any(axis=1)].head()

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


* document 칼럼
    * Null이 일부 존재.
    * 문자가 아닌 숫자의 경우 단어적인 의미로 부족.
    * 정규 표현식을 사용하여 공백으로 변환.

In [25]:
import re

df_train = df_train.fillna(' ')
# \d : 숫자
df_train['document'] = df_train['document'].apply(lambda x: re.sub(r'\d+', " ", x))

df_test = pd.read_csv('/content/drive/MyDrive/머신러닝 교과서 with 파이썬, 사이킷런, 텐서플로/data/ratings_test.txt', delimiter='\t')
df_test = df_test.fillna(' ')
df_test['document'] = df_test['document'].apply(lambda x: re.sub(r'\d+', " ", x))

# id 칼럼 삭제
df_train.drop('id', axis=1, inplace=True)
df_test.drop('id', axis=1, inplace=True)

In [26]:
if not os.path.isfile('NonNum_okt_train.npz'):
    tfidf = TfidfVectorizer(ngram_range=(1, 2), 
                            min_df=3,
                            max_df=0.9,
                            tokenizer=okt.morphs, 
                            token_pattern=None)
    tfidf.fit(df_train['document'])
    X_train_NonNum_okt = tfidf.transform(df_train['document'])
    X_test_NonNum_okt = tfidf.transform(df_test['document'])
    save_npz('NonNum_okt_train.npz', X_train_NonNum_okt)
    save_npz('NonNum_okt_test.npz', X_test_NonNum_okt)
else:
    X_train_NonNum_okt = load_npz('NonNum_okt_train.npz')
    X_test_NonNum_okt = load_npz('NonNum_okt_test.npz')

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

lg_clf = LogisticRegression(random_state=0)

# 파라미터 C 최적화를 위해 GridSearchCV를 이용
params = {'C': [1.0, 3.5, 4.5, 5.5, 10.0, 50.0, 100.0]}
grid_cv = GridSearchCV(lg_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(X_train_NonNum_okt, df_train['label'])
print(grid_cv.best_score_)
print(grid_cv.best_params_)

Fitting 3 folds for each of 7 candidates, totalling 21 fits
0.8592933333333334
{'C': 3.5}


In [28]:
from sklearn.metrics import accuracy_score

best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(X_test_NonNum_okt)
print(accuracy_score(df_test['label'], preds))

0.86186


In [29]:
sgd_clf = SGDClassifier(loss='log', random_state=1) # 손실 함수는 로지스틱 손실('log')을 사용, 다른 손실함수를 매개변수 탐색에 포함할 수 있다.
param_dist = {'alpha':loguniform(0.0001, 100.0)} # 규제를 위한 매개변수, RandomizedSearchCV 클래스를 사용하기 위해 loguniform 함수로 탐색 번위 지정
random_cv = RandomizedSearchCV(estimator=sgd_clf,
                             param_distributions=param_dist,
                             n_iter=50, # 총 반복횟수는 50회
                             random_state=1,
                             verbose=1)
random_cv.fit(X_train_NonNum_okt, df_train['label'])

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc2a1181e50>},
                   random_state=1, verbose=1)

In [30]:
print(random_cv.best_score_)
print(random_cv.best_params_)

0.8214266666666667
{'alpha': 0.0001001581395585897}


In [31]:
best_estimator = random_cv.best_estimator_
preds = best_estimator.predict(X_test_NonNum_okt)
print(accuracy_score(df_test['label'], preds))

0.8152
